# Практическая работа к уроку №3

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

X_st = X.T.copy()
X_st[:, 1] = calc_std_feat(X_st[:, 1])
X_st[:, 2] = calc_std_feat(X_st[:, 2])
X_st[:, 3] = calc_std_feat(X_st[:, 3])

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [3]:
def calc_logloss(y, y_pred, eps=1e-8):
    err = - np.mean(y * np.log(y_pred + eps) + (1.0 - y) * np.log(1.0 - y_pred + eps))
    return err

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [4]:
def eval_model(X, y, iterations, alpha=1e-1):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations+1):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1 / n * np.dot((y_pred - y).T, X))
        if i % 5000 == 0:
            print(i, W, err)
    return W

In [5]:
W = eval_model(X_st, y, iterations=50000)

5000 [ 1.53535465 -4.5908596  -3.5151729   9.41836954] 0.22241203642356439
10000 [ 2.48428253 -6.06769399 -5.34752279 13.29132815] 0.17913712050019812
15000 [ 3.17728368 -7.19574836 -6.70750681 16.21354578] 0.15473106260179312
20000 [ 3.72202873 -8.106698   -7.79591191 18.56784588] 0.13897337883019217
25000 [ 4.17132283 -8.87071374 -8.70905565 20.54705365] 0.12787657151420617
30000 [ 4.55467337 -9.52943786 -9.50100795 22.26234911] 0.11956367991544845
35000 [  4.89003336 -10.10931007 -10.20483817  23.78299785] 0.11304375327266888
40000 [  5.18905278 -10.62804616 -10.84208468  25.15475706] 0.10774689728535938
45000 [  5.45968273 -11.09804819 -11.42747681  26.40928677] 0.10332284168408164
50000 [  5.70757363 -11.5283176  -11.97148528  27.569326  ] 0.09954459557420335


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [6]:
def calc_pred_proba(W, X):
    z = np.dot(X, W)
    y_pred_proba = sigmoid(z)
    return y_pred_proba

y_proba = calc_pred_proba(W, X_st)
y_proba

array([0.33472167, 0.00359485, 1.        , 0.04086419, 0.9489814 ,
       0.01841023, 1.        , 0.01510532, 0.63356234, 0.99999999])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [7]:
def calc_pred(W, X, treshhold=0.5):
    z = np.dot(X, W)
    y_pred_proba = sigmoid(z)
    y_pred = list(map(lambda x: 1 if x > treshhold else 0, y_pred_proba))
    return y_pred

y_pred = calc_pred(W, X_st)
y_pred

[0, 0, 1, 0, 1, 0, 1, 0, 1, 1]

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [8]:
def metrics(y, y_pred, print_res=True, beta=1):
    assert len(y) == len(y_pred), "dim y not equal y_pred"
    
    TP, TN, FP, FN = 0, 0, 0, 0
    for i in range(len(y)):
        if y[i] == y_pred[i]:
            if y[i] == 1:
                TP += 1
            else:
                TN += 1
        else:
            if y[i] == 1:
                FN += 1
            else:
                FP += 1
    
    conf_matrix = pd.DataFrame(data=[[TP, FP], [FN, TN]],
                               columns=["y=1", "y=0"],
                               index=["y_pred=1", "y_pred=0"])
    accuracy = np.mean(y == y_pred)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f_score = (1 + beta**2) / beta**2 * precision * recall / (precision + recall)

    if print_res:
        print(f"accuracy: {accuracy}\n"\
        f"precision: {precision}\n"\
        f"recall: {recall}\n"\
        f"f_score: {f_score}\n"\
        f"confusion matrix\n{conf_matrix}")

    return accuracy, precision, recall, f_score, conf_matrix

In [9]:
accuracy, precision, recall, f_score, conf_matrix = metrics(y, y_pred)

accuracy: 1.0
precision: 1.0
recall: 1.0
f_score: 1.0
confusion matrix
          y=1  y=0
y_pred=1    5    0
y_pred=0    0    5


6. Могла ли модель переобучиться? Почему?

Дать точного ответа на этот вопрос нельзя, так как у нас нет валидационной выборки, чтобы сравнить качество валидационной выборки с тестовой.

Но здесь все же модель явно переобучилась, так как мы получили 100% качество классификации.

7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [10]:
def eval_model_l1(X, y, iterations, l1_lambda, alpha=1e-2):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations+1):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred) + np.sum(np.abs(W))
        W -= alpha * (1 / n * np.dot((y_pred - y).T, X)) \
            + l1_lambda * np.sign(W)
        if i % 5000 == 0:
            print(i, W, err)
    return W

In [11]:
W_l1 = eval_model_l1(X_st, y, iterations=50000, l1_lambda=0.001)

5000 [-4.42913855e-04 -1.74762829e-03  1.89916063e-04  3.77738832e-01] 1.0171242352110201
10000 [-5.01021000e-04 -1.56650076e-03 -4.37683093e-05  3.77120113e-01] 1.0183324436214845
15000 [-5.62605885e-04 -8.48559863e-04  2.81003097e-04  3.77123865e-01] 1.0172933989256183
20000 [-6.25581977e-04 -1.37969738e-04  5.97760274e-04  3.77121808e-01] 1.0162626090326734
25000 [-0.00069009 -0.00143082 -0.00108773  0.3771165 ] 1.017406711602413
30000 [-0.00075643 -0.00070606 -0.00075555  0.37712699] 1.017024710492117
35000 [-8.24106887e-04  5.93944714e-07 -4.42104975e-04  3.77122380e-01] 1.0169523942404277
40000 [-8.93524951e-04 -1.28870137e-03 -1.23940506e-04  3.77120743e-01] 1.0177614406840092
45000 [-9.64658052e-04 -5.70538245e-04  2.00662523e-04  3.77123776e-01] 1.0167121990506502
50000 [9.77027066e-04 1.40628308e-04 5.19792295e-04 3.77123800e-01] 1.0157434663183058


In [12]:
y_pred_l1 = calc_pred(W_l1, X_st)
accuracy, precision, recall, f_score, conf_matrix = metrics(y, y_pred_l1)

accuracy: 0.8
precision: 0.8
recall: 0.8
f_score: 0.8000000000000002
confusion matrix
          y=1  y=0
y_pred=1    4    1
y_pred=0    1    4


In [13]:
def eval_model_l2(X, y, iterations, l2_lambda, alpha=1e-2):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations+1):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred) + np.sum(W **2)
        W -= alpha * (1 / n * np.dot((y_pred - y).T, X)) \
            + l2_lambda * 2 * W
        if i % 5000 == 0:
            print(i, W, err)
    return W

In [14]:
W_l2 = eval_model_l2(X_st, y, iterations=50000, l2_lambda=0.001)

5000 [ 0.00095209 -0.25612807 -0.0222485   0.56346989] 0.9850570042726269
10000 [ 0.00095207 -0.25612372 -0.02224999  0.56346698] 0.9850520957077217
15000 [ 0.00095207 -0.25612372 -0.02224999  0.56346698] 0.9850520956694253
20000 [ 0.00095207 -0.25612372 -0.02224999  0.56346698] 0.9850520956694253
25000 [ 0.00095207 -0.25612372 -0.02224999  0.56346698] 0.9850520956694253
30000 [ 0.00095207 -0.25612372 -0.02224999  0.56346698] 0.9850520956694253
35000 [ 0.00095207 -0.25612372 -0.02224999  0.56346698] 0.9850520956694253
40000 [ 0.00095207 -0.25612372 -0.02224999  0.56346698] 0.9850520956694253
45000 [ 0.00095207 -0.25612372 -0.02224999  0.56346698] 0.9850520956694253
50000 [ 0.00095207 -0.25612372 -0.02224999  0.56346698] 0.9850520956694253


In [15]:
y_pred_l2 = calc_pred(W_l2, X_st)
accuracy, precision, recall, f_score, conf_matrix = metrics(y, y_pred_l2)

accuracy: 0.8
precision: 0.8
recall: 0.8
f_score: 0.8000000000000002
confusion matrix
          y=1  y=0
y_pred=1    4    1
y_pred=0    1    4
